# 누락 데이터(결측값) 처리 
#### [ Pandas 누락데이터 NaN]
#### [ NaN, Not a Number ]
#### [ Pandas 에서 NaN, None ]
#### [ 널(결측값)값 연산하기 ]
#### [ 결측값 찾기]
#### [ 결측값 제거하기 ]
#### [ 결측값 채우기 ]



- 누락데이터 표현 : null, Nan, NA
- 누락데이터 처리
  - 마스크 사용
  - 센티널값(데이터의 종료를 나타내기 위해 사용하는 특수한 값)을 사용

In [1]:
import pandas as pd
import numpy as np

print("pandas ver : ",pd.__version__)
print("numpy ver : ",np.__version__)

pandas ver :  0.24.2
numpy ver :  1.16.4


### [ Pandas 누락데이터 NaN]

- Pandas에서는 사용한 첫번재 센티널 값은 None(파이썬 객체임)을 사용
- None은 파이썬 객체의 배열에서만 사용 할 수 있음
- Numpy, Pandas 배열에서는 사용 안됨
  - **None이 들어간 파이선 리스트**로 **Series**를 생성하면 None 데이터는 **NaN으로 변경됨**

In [12]:
# Python 리스트에 None을 넣어 리스트 선언
listData = [1,2,None,3]
# list를 이용하여 Series선언
ser = pd.Series(listData)
print("listData : \n", listData)
print(type(listData))
print("\n")
print("Series ser data: \n",ser)
print(type(ser))

listData : 
 [1, 2, None, 3]
<class 'list'>


Series ser data: 
 0    1.0
1    2.0
2    NaN
3    3.0
dtype: float64
<class 'pandas.core.series.Series'>


In [26]:
# Numpy로 생성
vals1 = np.array(listData)
vals1

array([1, 2, None, 3], dtype=object)

In [32]:
vals2 = np.array([1,2,3,None])
vals2

array([1, 2, 3, None], dtype=object)

In [25]:
print("Numpy vals1 data : \n",vals1)
print(type(vals1))
print("Numpy vals2 data : \n",vals2)
print(type(vals1))

Numpy vals1 data : 
 [1 2 None 3]
<class 'numpy.ndarray'>
Numpy vals2 data : 
 [1 2 3]
<class 'numpy.ndarray'>


- None은 파이썬 객체이므로 Numpy/Pandas 배열에서 사용 할 수 없음
- 데이터타입(dtype)이 'Object'인 배열에서만 사용 살 수 있음

In [34]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    # 1E6(1.0e+6) : 1,000,000
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
63.3 ms ± 1.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
747 µs ± 7.94 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)



In [37]:
# 배열에 None가 있을 경우 Sum, Mean 연산시 에러가 발생함
vals1.sum()
# None이 계산을  할 수 없는 NoneType 객체 이기 때문에 ...

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

### [ NaN, Not a Number ]

- 누락 데이터 표현 : 표준 IEEE 부동 소수점 표기 방법 

In [41]:
print(np.nan)
print(type(np.nan))

nan
<class 'float'>


In [43]:
print(None)
print(type(None))

None
<class 'NoneType'>


- python의 None 객체와 Numpy의 NaN 객체의 차이
  - None 의 타입 : NontType(dtype : object)
  - NaN 의 타입 : float(부동 소수점)
  
- **NaN 은 부동 소수점 타입으로 배열에서 연산을 할 수 있음**
- NaN 과 연산이 된 결과는 모두 NaN이 됨

In [52]:
# NaN 과 연산
print(np.nan + 1)
print(np.nan * 10)

nan
nan


In [53]:
# np.nan 을 사용하여 결측값이 들어간 배열 선언
arr = np.array([1,2,np.nan,3])
print(arr)
print(type(arr))

[ 1.  2. nan  3.]
<class 'numpy.ndarray'>


In [54]:
print(arr.sum())
print(arr.min())
print(arr.max())

nan
nan
nan


### [ Pandas 에서 NaN, None ]

- Pandas 에서는 None가 입력되면 NaN으로 변환

In [56]:
ser = pd.Series([1, np.nan, 2, 3, None])
print("ser : \n",ser)

ser : 
 0    1.0
1    NaN
2    2.0
3    3.0
4    NaN
dtype: float64


- NaN 값이 부동소수점 타입이기 때문에 기존의 정수형 데이터 모두 부동소수점으로 자동으로 상향 변환 됨

###### Pandas 타입별 NA 값 처리 방식 

The following table lists the upcasting conventions in Pandas when NA values are introduced:

|타입클래스       | NA 값을 저장할 때의 변환         | NA 센티널 값             |
|--------------|-----------------------------|------------------------|
| ``floating`` | No change                   | ``np.nan``             |
| ``object``   | No change                   | ``None`` or ``np.nan`` |
| ``integer``  | Cast to ``float64``         | ``np.nan``             |
| ``boolean``  | Cast to ``object``          | ``None`` or ``np.nan`` |

Keep in mind that in Pandas, string data is always stored with an ``object`` dtype.

### [ 널(결측값)값 연산하기 ]

- Pandas 에서는 None 가 NaN을 호환되게 함
- Pandas에서는 데이터 구조에서 널(Null)값을 감지하고 삭제 하는 메서드 제공

- ``isnull()``: 누락된 값을 가리키는 부울마스크 생성
- ``notnull()``: ``isnull()``의 반대, 널인 아닌것을 찾음
- ``dropna()``: 데이터에 필터를 적용한 버전을 반환
- ``fillna()``: 누락값을 채우거나 전가된 데이터 사본을 반환

### [ 결측값 찾기]

In [58]:
data = pd.Series([1, np.nan, 'hello', None, False])
print("data (series) :  \n",data)
print(type(data))

data (series) :  
 0        1
1      NaN
2    hello
3     None
4    False
dtype: object
<class 'pandas.core.series.Series'>


- `isnull()`

In [69]:
print(data.isnull())

0    False
1     True
2    False
3     True
4    False
dtype: bool


- `notnull()`

In [70]:
print(data.notnull())

0     True
1    False
2     True
3    False
4     True
dtype: bool


- 마스킹을 통한 결측값 확인

In [71]:
# 마스킹하여 결측값을 제거하고 나머지 데이터를 출력함 => 결측값을 제거하고 나머지 출력!!
print(data[data.notnull()])

0        1
2    hello
4    False
dtype: object


### [ 결측값 제거하기 ]

- 결측값 제거
  - `dropna()` : 결측값을 제거한 데이터를 반환

- **Series 결측값 제거**

In [75]:
# series 결측값 제거
print(data)
result = data.dropna()
print("\n")
print("result : \n",result)
print("type : ",type(result))
print("\n")
print("data : \n",data)

0        1
1      NaN
2    hello
3     None
4    False
dtype: object


result : 
 0        1
2    hello
4    False
dtype: object
type :  <class 'pandas.core.series.Series'>


data : 
 0        1
1      NaN
2    hello
3     None
4    False
dtype: object


- **Dataframe 결측값 제거**
  - 단일값만 삭제 할 수 없음
  - 결측값을 포함한 행이나 열 전체 삭제 가능

In [151]:
# Dataframe 결측값 제거
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
print(df)
print(type(df))

     0    1  2
0  1.0  NaN  2
1  2.0  3.0  5
2  NaN  4.0  6
<class 'pandas.core.frame.DataFrame'>


In [152]:
# 결측값이 들어간 모든 행 삭제
result_df = df.dropna()
print(result_df)

     0    1  2
1  2.0  3.0  5


In [153]:
# 결측값이 들어간 열 삭제
result_df2 = df.dropna(axis=1)
print(result_df2)

   2
0  2
1  5
2  6


In [154]:
# axis='columns'로 입력 가능함 (행을 삭제시 axis를 입력 하지 않음)
df.dropna(axis='columns')

,2
0,2
1,5
2,6


###### Dataframe에서 결측값을 제거하면 실제 사용가능한 데이터도 모두 삭제됨

- `DataFrame.dropna(self, axis=0, how='any', thresh=None, subset=None, inplace=False)`
  - axis : {0 or ‘index’, 1 or ‘columns’}, default 0
  - how : {‘any’, ‘all’}, default ‘any’
  - thresh : int, optional. Require that many non-NA values.  

In [155]:
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [156]:
# columns = 3 인 데이터를 결측값으로 초기화 하여 추가
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


- **how=all** : 컬럼이나 열이 **모두** 결측값일때 삭제 

In [157]:
# 컬럼 모두가 결측값 일때만 삭제 : axis='columns', how='all'
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [158]:
# index 3에 결측값으로 초기화 하여 추가 
df.loc[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2.0,NaN
1,2.0,3.0,5.0,NaN
2,NaN,4.0,6.0,NaN
3,NaN,NaN,NaN,NaN


In [159]:
# 행 모두가 결측값 일때만 삭제 : how='all'
df.dropna(how='all')

,0,1,2,3
0,1.0,NaN,2.0,NaN
1,2.0,3.0,5.0,NaN
2,NaN,4.0,6.0,NaN


In [162]:
# # 행 모두가 결측값 일때만 삭제 : axis='rows', how='all'
df.dropna(axis='rows', how='all')

,0,1,2,3
0,1.0,NaN,2.0,NaN
1,2.0,3.0,5.0,NaN
2,NaN,4.0,6.0,NaN


- **thresh 매개변수**를 이용하여 결측값 제거할때 최소 갯수를 정해줌(thresh=2 이면 2개 이상의 결측값이 있을때 삭제)

In [166]:
df

,0,1,2,3
0,1.0,NaN,2.0,NaN
1,2.0,3.0,5.0,NaN
2,NaN,4.0,6.0,NaN
3,NaN,NaN,NaN,NaN


In [167]:
df.dropna(axis='rows', thresh=2)

,0,1,2,3
0,1.0,NaN,2.0,NaN
1,2.0,3.0,5.0,NaN
2,NaN,4.0,6.0,NaN


In [168]:
df.dropna(axis='columns', thresh=2)

,0,1,2
0,1.0,NaN,2.0
1,2.0,3.0,5.0
2,NaN,4.0,6.0
3,NaN,NaN,NaN


### [ 결측값 채우기 ]

- 결측값을 삭제 하지않고 유요한 값으로 대체해야 할때 사용
- `isnull()`, `fillna()`제공

- **Series 에서 결측값 채우기**

In [169]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
print(data)
print(type(data))

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64
<class 'pandas.core.series.Series'>


- `fillna(n)` : n 값으로 결측값 모두를 채움
- `Series.fillna(self, value=None, method=None, axis=None, inplace=False, limit=None, downcast=None, **kwargs)`
  - method : {‘backfill’, ‘bfill’, ‘pad’, ‘ffill’, None}, default None. Method to use for filling holes in reindexed Series pad / ffill: propagate last valid observation forward to next valid backfill / bfill: use next valid observation to fill gap.


In [173]:
# 0으로 결측값 채움
print(data.fillna(0))

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64


- `fillna(method='ffill')` : 이전 인덱스의 값으로 채움 

In [174]:
print(data)
print("\n")
print(data.fillna(method='ffill'))

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64


a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64


- `fillna(method='bfill')` : 다음 인덱스 값으로 채움

In [175]:
print(data)
print("\n")
print(data.fillna(method='bfill'))

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64


a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64


- **Dataframe 에서 결측값 채우기**
- `DataFrame.fillna(self, value=None, method=None, axis=None, inplace=False, limit=None, downcast=None, **kwargs)`
  - method : {‘backfill’, ‘bfill’, ‘pad’, ‘ffill’, None}, default None. Method to use for filling holes in reindexed Series pad / ffill: propagate last valid observation forward to next valid backfill / bfill: use next valid observation to fill gap.

In [176]:
print(df)

     0    1    2   3
0  1.0  NaN  2.0 NaN
1  2.0  3.0  5.0 NaN
2  NaN  4.0  6.0 NaN
3  NaN  NaN  NaN NaN


In [177]:
df.fillna(method='ffill', axis='columns')

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0
3,NaN,NaN,NaN,NaN


In [178]:
print(df)

     0    1    2   3
0  1.0  NaN  2.0 NaN
1  2.0  3.0  5.0 NaN
2  NaN  4.0  6.0 NaN
3  NaN  NaN  NaN NaN


In [179]:
df.fillna(method='bfill', axis='rows')

,0,1,2,3
0,1.0,3.0,2.0,NaN
1,2.0,3.0,5.0,NaN
2,NaN,4.0,6.0,NaN
3,NaN,NaN,NaN,NaN


In [180]:
df.fillna(method='pad', axis='columns')

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0
3,NaN,NaN,NaN,NaN
